In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical



In [2]:
try:
    import dill as pickle
except ImportError:
    import pickle
    
    
def load_doc(filename): 
    file = open(filename, 'r') 
    text = file.read() 
    file.close()
    return text



In [3]:
filename = "proba.txt"
# load descriptions
doc = load_doc(filename)
print(doc)




10-Yard Fight - square7.png : 0-0-65-102-26 393-0-65-0-26 393-0-69-102-26 524-0-69-0-26 524-0-65-102-26
1943 - square3.png : 69-0-66-63-56 69-1-59-63-56 69-2-54-54-58 69-3-49-65-47 69-4-71-78-48
1999 - square2.png : 0-0-64-120-NONE 0-1-59-120-NONE 0-2-45-120-NONE 120-0-64-0-12 120-0-64-120-12
3D Battles of Worldrunner, The - square0.png : 0-2-59-45-83 0-1-43-79-34 0-0-79-79-81 166-2-59-0-83 166-1-43-0-34
3D Galax - square0.png : 0-3-82-81-94 0-1-73-87-18 0-1-70-87-18 0-1-77-87-18 0-5-61-73-92
3D StarFighter - square11.png : 0-9-35-100-NONE 0-9-36-100-NONE 0-9-40-100-NONE 0-9-38-100-NONE 27-9-38-0-NONE
4 Nin Shogi (Japan) - square6.png : 0-2-33-100-33 0-3-81-100-11 0-9-44-115-0 0-9-39-91-0 0-9-35-91-0
4-in-1 Fun Pak - square6.png : 0-0-45-100-87 0-1-33-100-33 0-4-69-76-81 0-6-52-74-48 0-6-45-74-48
4d Boxing - square2.png : 0-0-54-95-29 0-0-49-95-29 0-2-66-96-103 0-9-52-82-0 0-9-42-82-0
4th & Inches - square0.png : 0-0-40-110-48 0-2-40-110-47 0-3-40-110-27 0-6-40-110-17 96-1-47-110-15
7 

In [7]:
def load_descriptions(doc): #todo
    mapping = dict()
    
    for line in doc.split('\n'):
        if len(line) < 2:
            continue
        tokens = line.split(" : ")
        
        image_id, image_desc = tokens[0], tokens[1:]
        
        image_id =image_id.split('.')[0]
        
        image_desc = ' '.join(image_desc)
        
        if image_id not in mapping:
            mapping[image_id] = list()
        
        mapping[image_id].append(image_desc)
    return mapping

descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))


Loaded: 52 


In [10]:

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# def clean_descriptions(descriptions):
    
#     table = str.maketrans('', '', string.punctuation)
#     for key, desc_list in descriptions.items():
#         for i in range(len(desc_list)):
#             desc = desc_list[i]
            
#             desc = tokenizer.tokenize(desc)
#             print(desc)
#             desc = [word.lower() for word in desc]
            
#             desc = [word for word in desc if len(word)>1]
            
#             desc_list[i] =  ' '.join(desc)

# clean_descriptions(descriptions)

def getVocabulary(descriptions):
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split(" ")) for d in descriptions[key]]
    return all_desc

# summarize vocabulary
vocabulary = getVocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))
print(vocabulary)



Original Vocabulary Size: 244
{'27-9-38-0-NONE', '192-0-48-0-99', '288-0-47-0-80', '0-9-40-100-NONE', '225-0-62-80-0', '0-9-40-70-0', '0-1-73-87-18', '0-4-82-127-49', '98-4-48-45-57', '0-1-57-100-41', '0-0-48-79-80', '0-6-40-110-17', '256-0-59-0-6', '0-0-71-100-80', '383-1-64-127-81', '0-2-53-83-32', '1358-1-40-104-33', '0-9-44-115-0', '0-0-98-110-11', '0-0-47-100-80', '96-1-47-110-15', '271-3-63-0-4', '107-9-40-48-0', '0-0-70-110-28', '235-1-65-0-81', '0-2-40-110-47', '514-0-52-0-12', '0-1-67-110-28', '1350-9-36-94-0', '4015-0-60-40-87', '256-0-59-105-6', '0-1-66-100-50', '288-0-71-0-80', '0-3-80-84-17', '0-9-35-79-0', '0-2-66-96-103', '0-9-47-48-0', '0-9-35-99-0', '0-0-79-79-81', '133-0-75-100-80', '0-5-60-79-48', '0-0-59-105-6', '0-9-49-99-0', '0-2-33-109-33', '75-1-65-0-81', '0-3-45-102-118', '166-1-43-0-34', '0-1-70-87-18', '0-0-49-95-29', '80-2-43-0-57', '98-0-29-81-48', '0-0-62-67-61', '4302-0-65-40-87', '0-9-38-99-0', '112-9-40-87-25', '0-9-52-82-0', '1350-9-55-0-0', '64-0-38-0

In [11]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

save_descriptions(descriptions, 'descriptions.txt')
print("Desciption.txt created")


Desciption.txt created


In [13]:
# load Train Data
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    
    for line in doc.split('\n'):
        
        if len(line) < 1:
            continue
 
        dataset.append(line)
    return (dataset)
 
filename = 'train_images.txt'
train_names = load_set(filename)
print('Train size: %d' % len(train_names))

Directroy_path = '/textImages/'
train_img= [Directroy_path+i  for i in train_names]


Train size: 52


In [14]:
filename = 'test_images.txt'
test_names = load_set(filename)
print('Test Size: %d' % len(test_names))

Directroy_path = '/testImages/'
test_img= [Directroy_path+i  for i in test_names]






FileNotFoundError: [Errno 2] No such file or directory: 'test_images.txt'

In [19]:

def load_clean_descriptions(filename, dataset):
    # load document
    dataset = [i.split('.')[0] for i in dataset]
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        
        tokens = line.split(".png")
        
        image_id, image_desc = tokens[0], tokens[1].split(" ")
        if image_id in dataset:
          
            if image_id not in descriptions:
                descriptions[image_id] = list()
            
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            
            descriptions[image_id].append(desc)
    return descriptions

# descriptions
train_descriptions = load_clean_descriptions('proba2.txt', train_names)

print('Descriptions: train=%d' % len(train_descriptions))


10-Yard Fight - square7
1943 - square3
1999 - square2
3D Battles of Worldrunner, The - square0
3D Galax - square0
3D StarFighter - square11
4 Nin Shogi (Japan) - square6
4-in-1 Fun Pak - square6
4d Boxing - square2
4th & Inches - square0
7 Gates Of Jambala - square0
7 Gates Of Jambala - square3
7th Saga - square10
7th Saga - square16
8 Eyes - square2
9 Lives - square12
90 Minutes_ European Prime Goal - square6
A Boy and His Blob - square0
A Bug's Life - square0
A Nightmare on Elm Street - square10
A Prehistoric Tale - square0
Abadox - square8
Ace Combat - square0
Ace Combat 04_ Shattered Skies - square0
Ace Combat 2 - square2
Action 52 - square1
Action Biker - square2
ActRaiser - square0
ActRaiser 2 - square15
Addams Family, The_ Pugsley's Scavenger Hunt - square1
Advance Guardian Heroes - square0
Advance Wars - square4
Advance Wars 2_ Black Hole Rising - square16
Advance Wars_ Days of Ruin - square1
Advance Wars_ Dual Strike - square10
Advanced Tactical Fighter - square0
Advanced V  G

In [40]:
with open("train_descriptions.pickle", "wb") as encoded_pickle:
    pickle.dump(train_descriptions, encoded_pickle)
    

In [34]:
    
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    
    x = preprocess_input(x)
    return x


model = InceptionV3(weights='imagenet')
model.summary()

model_new = Model(model.input, model.layers[-2].output)

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 149, 149, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 149, 149, 32) 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 149, 149, 32) 0           batch_normalization_470[0][0]    
_______________________________________________________________________________________

In [35]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) 
    v = model_new.predict(image) 
    v = np.reshape(v, v.shape[1]) 
    return v


In [36]:
start = time()
k=0
encoded_train = {}
for img in tqdm(train_img):
    img = "C:/soft/bb/MusicGenerator"+img
    encoded_train[img[len(Directroy_path):]] = encode(img)


<ipython-input-36-671fede39f70>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for img in tqdm(train_img):


  0%|          | 0/52 [00:00<?, ?it/s]

In [39]:


# Save Train features
with open("encoded_train_images.pickle", "wb") as encoded_pickle:
    pickle.dump(encoded_train, encoded_pickle)
    
    
# from pickle import dump, load
# with open("X.pickle", "wb") as encoded_pickle:
#     pickle.dump(encoded_test, encoded_pickle)
    
for key, c in encoded_train.items():
    print(key)
train_img[0][len(Directroy_path):]
len(encoded_train)

usicGenerator/textImages/10-Yard Fight - square7.png
usicGenerator/textImages/1943 - square3.png
usicGenerator/textImages/1999 - square2.png
usicGenerator/textImages/3D Battles of Worldrunner, The - square0.png
usicGenerator/textImages/3D Galax - square0.png
usicGenerator/textImages/3D StarFighter - square11.png
usicGenerator/textImages/4 Nin Shogi (Japan) - square6.png
usicGenerator/textImages/4-in-1 Fun Pak - square6.png
usicGenerator/textImages/4d Boxing - square2.png
usicGenerator/textImages/4th & Inches - square0.png
usicGenerator/textImages/7 Gates Of Jambala - square0.png
usicGenerator/textImages/7 Gates Of Jambala - square3.png 
usicGenerator/textImages/7th Saga - square10.png
usicGenerator/textImages/7th Saga - square16.png
usicGenerator/textImages/8 Eyes - square2.png
usicGenerator/textImages/9 Lives - square12.png
usicGenerator/textImages/90 Minutes_ European Prime Goal - square6.png 
usicGenerator/textImages/A Boy and His Blob - square0.png
usicGenerator/textImages/A Bug's 

52